# report

In [187]:
import pandas as pd
import os

In [83]:
file_list=['spy/'+i for i in os.listdir('spy')]

In [84]:
sample = pd.read_csv(file_list[0],skipinitialspace=True)

In [453]:
import pandas as pd
import os

def contract_selct(
    selct_type: str = 'main',
    contract_type: str = 'C',
    spy_spx : str = 'spy',
    date: str = None,
    main: pd.DataFrame = None,
    minAsk: float = None,
    maxDTE: int = 30,
    minDTE: int = 10,
    maxDistance: float = None,
    minDistance: float = None,
    maxTheta: float = None,
    minTheta: float = None,
    moneyness : str = 'out') ->pd.DataFrame:
    
    """
    return a dataframe containing the information of selected contracts
    
    input:
    selct_type: "main" or "hedge", declare what type of contracts we are searching, whether the main contracts we are shorting, or the contracts that we used to hedge
    contract_type: 'C' or 'P'
    date: str in form of "yyyymmdd", declare the date
    spy_spx : 'spy' or 'spx'
    main: None when selct_type = 'main', a dataframe containing the main contract when selct_type = 'hedge'
    minAsk: min contract ask price
    maxDTE: max DTE
    minDTE: min DTE
    maxDistance: max distance of strike from underlying in percentage
    minDistance: min distance of strike from underlying in percentage
    maxTheta: max Theta
    minTheta: min Theta
    moneyness: 'out' or 'in'
    """
    
    if maxDistance == None: maxDistance = 1000
    if minDistance == None: minDistance = 0
    if maxTheta == None: maxTheta = 0
    if minTheta == None: minTheta = -100
    if minAsk == None: minAsk = 0
    
    contract_pd = pd.DataFrame()
    contract_pd =  pd.read_csv(spy_spx+'_cleaned/'+spy_spx+'_eod_'+date[:6]+'.csv', 
                       index_col=['[QUOTE_DATE]','[EXPIRE_DATE]'], skipinitialspace=True)
    contract_pd = contract_pd.drop(columns='Unnamed: 0')
    contract_pd = contract_pd.iloc[contract_pd.index.get_level_values('[QUOTE_DATE]')==date[:4]+'-'+date[4:6]+'-'+date[6:]]
    contract_pd = contract_pd.loc[(contract_pd['[DTE]']>=minDTE) & (contract_pd['[DTE]']<=maxDTE) 
            & (contract_pd['[STRIKE_DISTANCE_PCT]']>=minDistance) & (contract_pd['[STRIKE_DISTANCE_PCT]']<=maxDistance)]
    if contract_type == 'C':
        contract_pd = contract_pd.drop(columns=['[P_BID]', '[P_ASK]', '[P_DELTA]', '[P_GAMMA]', '[P_VEGA]', '[P_THETA]'])
        if moneyness == 'out':
            contract_pd = contract_pd.loc[contract_pd['[STRIKE]'] >= contract_pd['[UNDERLYING_LAST]']]
            contract_pd = contract_pd.loc[(contract_pd['[C_THETA]'] >= minTheta) & (contract_pd['[C_THETA]'] <= maxTheta)]
        if moneyness == 'in':
            contract_pd = contract_pd.loc[contract_pd['[STRIKE]'] <= contract_pd['[UNDERLYING_LAST]']]
            contract_pd = contract_pd.loc[(contract_pd['[C_THETA]'] >= minTheta) & (contract_pd['[C_THETA]'] <= maxTheta)]
    if contract_type == 'P':
        contract_pd = contract_pd.drop(columns=['[C_BID]', '[C_ASK]', '[C_DELTA]', '[C_GAMMA]', '[C_VEGA]', '[C_THETA]'])
        if moneyness == 'out':
            contract_pd = contract_pd.loc[contract_pd['[STRIKE]'] <= contract_pd['[UNDERLYING_LAST]']]
            contract_pd = contract_pd.loc[(contract_pd['[P_THETA]'] >= minTheta) & (contract_pd['[P_THETA]'] <= maxTheta)]
        if moneyness == 'in':
            contract_pd = contract_pd.loc[contract_pd['[STRIKE]'] >= contract_pd['[UNDERLYING_LAST]']]
            contract_pd = contract_pd.loc[(contract_pd['[P_THETA]'] >= minTheta) & (contract_pd['[P_THETA]'] <= maxTheta)]
    
    contract_pd['spread_pct'] = (contract_pd['['+contract_type+'_ASK]']-contract_pd['['+contract_type+'_BID]'])*2/(contract_pd['['+contract_type+'_BID]']+contract_pd['['+contract_type+'_ASK]'])
    # Set inverted quote spread to zero
    contract_pd['spread_pct'][contract_pd['spread_pct'] < 0] = 0
    
    # filter out contract with very small ask
    contract_pd = contract_pd[contract_pd['['+contract_type+'_ASK]'] >= contract_pd['['+contract_type+'_ASK]'].mean()]
    contract_pd = contract_pd[contract_pd['['+contract_type+'_ASK]'] >= minAsk]
    
    for i in ['['+contract_type+'_DELTA]', '['+contract_type+'_GAMMA]', '['+contract_type+'_VEGA]', '['+contract_type+'_THETA]']:
        contract_pd['price_to_'+i]=(contract_pd['['+contract_type+'_BID]']+contract_pd['['+contract_type+'_ASK]'])/(2*contract_pd[i].abs())

        # utility function to select main contract    
    if selct_type == 'main':    
        if spy_spx == 'spx': contract_pd['utility'] = 5000*contract_pd['spread_pct'] - 0.5*contract_pd['price_to_['+contract_type+'_DELTA]'] -0.005*contract_pd['price_to_['+contract_type+'_GAMMA]'] - contract_pd['price_to_['+contract_type+'_VEGA]'] + 50* contract_pd['price_to_['+contract_type+'_THETA]']
        if spy_spx == 'spy': contract_pd['utility'] = 1000*contract_pd['spread_pct'] - 3*contract_pd['price_to_['+contract_type+'_DELTA]'] -contract_pd['price_to_['+contract_type+'_GAMMA]'] - 3* contract_pd['price_to_['+contract_type+'_VEGA]'] + 50* contract_pd['price_to_['+contract_type+'_THETA]']
    if selct_type == 'hedge':    
        if spy_spx == 'spx': contract_pd['utility'] = 5000*contract_pd['spread_pct'] + 0.05*contract_pd['price_to_['+contract_type+'_GAMMA]'] + 15*contract_pd['price_to_['+contract_type+'_VEGA]'] - 100* contract_pd['price_to_['+contract_type+'_THETA]']
        if spy_spx == 'spy': contract_pd['utility'] = 2000*contract_pd['spread_pct'] + 3*contract_pd['price_to_['+contract_type+'_GAMMA]'] + 3* contract_pd['price_to_['+contract_type+'_VEGA]'] - 100* contract_pd['price_to_['+contract_type+'_THETA]']
        
    contract_pd = contract_pd.drop(columns=['spread_pct', 'price_to_['+contract_type+'_DELTA]', 'price_to_['+contract_type+'_GAMMA]', 'price_to_['+contract_type+'_VEGA]', 'price_to_['+contract_type+'_THETA]'])

    return contract_pd.sort_values(by=['utility']).head(3)
    # return contract_pd.sort_values(by=['utility'])

In [456]:
contract_selct(date= '20190913', spy_spx='spx', contract_type='P', minAsk=0.2, minDistance = 0.08)

[UNDERLYING_LAST]  [DTE]  [STRIKE]  [P_BID]  \
[QUOTE_DATE] [EXPIRE_DATE]                                                
2019-09-13   2019-10-11               3007.01   28.0    2765.0     4.80   
             2019-10-11               3007.01   28.0    2750.0     4.30   
             2019-10-11               3007.01   28.0    2760.0     4.60   
             2019-10-11               3007.01   28.0    2730.0     3.70   
             2019-10-11               3007.01   28.0    2755.0     4.40   
...                                       ...    ...       ...      ...   
             2019-10-02               3007.01   19.0    2625.0     0.74   
             2019-10-11               3007.01   28.0    2480.0     0.60   
             2019-09-25               3007.01   12.0    2725.0     0.70   
             2019-09-23               3007.01   10.0    2765.0     0.65   
             2019-10-02               3007.01   19.0    2610.0     0.64   

                            [P_ASK]  [P_DELTA]  [P_GAMMA]  [P_VEGA]  \
[QUOTE_DATE] [EXPIRE_DATE]                                            
2019-09-13   2019-10-11        4.91   -0.06468    0.00080   1.07542   
             2019-10-11        4.41   -0.05831    0.00071   0.98295   
             2019-10-11        4.80   -0.06282    0.00070   1.04561   
             2019-10-11        3.81   -0.05015    0.00055   0.86868   
             2019-10-11        4.60   -0.06013    0.00065   1.01177   
...                             ...        ...        ...       ...   
             2019-10-02        0.90   -0.01237    0.00016   0.22966   
             2019-10-11        0.76   -0.00841    0.00013   0.18903   
             2019-09-25        0.86   -0.01570    0.00028   0.22162   
             2019-09-23        0.80   -0.01642    0.00030   0.21761   
             2019-10-02        0.80   -0.01077    0.00012   0.20506   

                            [P_THETA]  [STRIKE_DISTANCE]  \
[QUOTE_DATE] [EXPIRE_DATE]                                 
2019-09-13   2019-10-11      -0.37072              242.0   
             2019-10-11      -0.34664              257.0   
             2019-10-11      -0.36438              247.0   
             2019-10-11      -0.31543              277.0   
             2019-10-11      -0.35423              252.0   
...                               ...                ...   
             2019-10-02      -0.14488              382.0   
             2019-10-11      -0.09060              527.0   
             2019-09-25      -0.19538              282.0   
             2019-09-23      -0.20989              242.0   
             2019-10-02      -0.13103              397.0   

                            [STRIKE_DISTANCE_PCT]  spread_pct  \
[QUOTE_DATE] [EXPIRE_DATE]                                      
2019-09-13   2019-10-11                     0.080    0.022657   
             2019-10-11                     0.085    0.025258   
             2019-10-11                     0.082    0.042553   
             2019-10-11                     0.092    0.029294   
             2019-10-11                     0.084    0.044444   
...                                           ...         ...   
             2019-10-02                     0.127    0.195122   
             2019-10-11                     0.175    0.235294   
             2019-09-25                     0.094    0.205128   
             2019-09-23                     0.080    0.206897   
             2019-10-02                     0.132    0.222222   

                            price_to_[P_DELTA]  price_to_[P_GAMMA]  \
[QUOTE_DATE] [EXPIRE_DATE]                                           
2019-09-13   2019-10-11              75.061843         6068.750000   
             2019-10-11              74.687018         6133.802817   
             2019-10-11              74.816937         6714.285714   
             2019-10-11              74.875374         6827.272727   
             2019-10-11              74.837851         6923.076923   
...        

In [450]:
data= 'spy'
date= '20170912'
maxDTE: int = 30,
minDTE: int = 10,
maxDistance: float = 0.2,
minDistance: float = 0.08,
maxTheta: float = -0.0,
minTheta: float = -1.2,
contract_type: str = 'C'
moneyness = 'out'

contract_pd =  pd.read_csv(data+'_cleaned/'+data+'_eod_'+date[:6]+'.csv', 
                   index_col=['[QUOTE_DATE]','[EXPIRE_DATE]'], skipinitialspace=True)
contract_pd = contract_pd.drop(columns='Unnamed: 0')
contract_pd = contract_pd.iloc[contract_pd.index.get_level_values('[QUOTE_DATE]')==date[:4]+'-'+date[4:6]+'-'+date[6:]]
contract_pd = contract_pd.loc[(contract_pd['[DTE]']>=minDTE) & (contract_pd['[DTE]']<=maxDTE) 
                & (contract_pd['[STRIKE_DISTANCE_PCT]']>=minDistance) & (contract_pd['[STRIKE_DISTANCE_PCT]']<=maxDistance)]

In [354]:
if contract_type == 'C':
    contract_pd = contract_pd.drop(columns=['[P_BID]', '[P_ASK]', '[P_DELTA]', '[P_GAMMA]', '[P_VEGA]', '[P_THETA]'])
    if moneyness == 'out':
        contract_pd = contract_pd.loc[contract_pd['[STRIKE]'] >= contract_pd['[UNDERLYING_LAST]']]
        contract_pd = contract_pd.loc[(contract_pd['[C_THETA]'] >= minTheta) & (contract_pd['[C_THETA]'] <= maxTheta)]
    if moneyness == 'in':
        contract_pd = contract_pd.loc[contract_pd['[STRIKE]'] <= contract_pd['[UNDERLYING_LAST]']]
        contract_pd = contract_pd.loc[(contract_pd['[C_THETA]'] >= minTheta) & (contract_pd['[C_THETA]'] <= maxTheta)]
if contract_type == 'P':
    contract_pd = contract_pd.drop(columns=['[C_BID]', '[C_ASK]', '[C_DELTA]', '[C_GAMMA]', '[C_VEGA]', '[C_THETA]'])
    if moneyness == 'out':
        contract_pd = contract_pd.loc[contract_pd['[STRIKE]'] <= contract_pd['[UNDERLYING_LAST]']]
        contract_pd = contract_pd.loc[(contract_pd['[P_THETA]'] >= minTheta) & (contract_pd['[P_THETA]'] <= maxTheta)]
    if moneyness == 'in':
        contract_pd = contract_pd.loc[contract_pd['[STRIKE]'] >= contract_pd['[UNDERLYING_LAST]']]
        contract_pd = contract_pd.loc[(contract_pd['[P_THETA]'] >= minTheta) & (contract_pd['[P_THETA]'] <= maxTheta)]

In [ ]:
contract_pd['spread_pct'] = (contract_pd['['+contract_type+'_ASK]']-contract_pd['['+contract_type+'_BID]'])*2/(contract_pd['['+contract_type+'_BID]']+contract_pd['['+contract_type+'_ASK]'])
contract_pd['spread_pct'][contract_pd['spread_pct'] < 0] = 0
contract_pd = contract_pd[contract_pd['['+contract_type+'_ASK]'] >= contract_pd['['+contract_type+'_ASK]'].mean()]

for i in ['['+contract_type+'_DELTA]', '['+contract_type+'_GAMMA]', '['+contract_type+'_VEGA]', '['+contract_type+'_THETA]']:
    contract_pd['price_to_'+i]=(contract_pd['['+contract_type+'_BID]']+contract_pd['['+contract_type+'_ASK]'])/(2*contract_pd[i].abs())

# utility function to select main contract    
contract_pd['utility'] = 1000*contract_pd['spread_pct'] - 3*contract_pd['price_to_['+contract_type+'_DELTA]'] -contract_pd['price_to_['+contract_type+'_GAMMA]'] - 3* contract_pd['price_to_['+contract_type+'_VEGA]'] + 50* contract_pd['price_to_['+contract_type+'_THETA]']

# contract_pd = contract_pd.drop(columns=['spread_pct', 'price_to_['+contract_type+'_DELTA]', 'price_to_['+contract_type+'_GAMMA]', 'price_to_['+contract_type+'_VEGA]', 'price_to_['+contract_type+'_THETA]'])

contract_pd.sort_values(by=['utility'])

In [369]:
contract_pd.tail(50)

[UNDERLYING_LAST]  [DTE]  [C_DELTA]  [C_GAMMA]  \
[QUOTE_DATE] [EXPIRE_DATE]                                                   
2017-09-12   2017-09-29                250.07   17.0    1.00000    0.00000   
             2017-09-29                250.07   17.0    1.00000    0.00000   
             2017-09-29                250.07   17.0    1.00000    0.00000   
             2017-09-29                250.07   17.0    1.00000    0.00000   
             2017-09-29                250.07   17.0    1.00000    0.00000   
             2017-09-29                250.07   17.0    1.00000    0.00000   
             2017-09-29                250.07   17.0    1.00000    0.00000   
             2017-09-29                250.07   17.0    1.00000    0.00000   
             2017-09-29                250.07   17.0    1.00000    0.00000   
             2017-09-29                250.07   17.0    0.00253    0.00107   
             2017-09-29                250.07   17.0    0.00241    0.00070   
             2017-09-29                250.07   17.0    0.00202    0.00056   
             2017-09-29                250.07   17.0    0.00136    0.00041   
             2017-09-29                250.07   17.0    0.00181    0.00039   
             2017-09-29                250.07   17.0    0.00121    0.00028   
             2017-09-29                250.07   17.0    0.00142    0.00029   
             2017-10-04                250.07   22.0    1.00000    0.00000   
             2017-10-04                250.07   22.0    0.95008    0.00527   
             2017-10-04                250.07   22.0    1.00000    0.00000   
             2017-10-04                250.07   22.0    1.00000    0.00000   
             2017-10-04                250.07   22.0    1.00000    0.00000   
             2017-10-04                250.07   22.0    1.00000    0.00000   
             2017-10-04                250.07   22.0    1.00000    0.00000   
             2017-10-04                250.07   22.0    1.00000    0.00000   
             2017-10-04                250.07   22.0    1.00000    0.00000   
             2017-10-04                250.07   22.0    1.00000    0.00000   
             2017-10-04                250.07   22.0    1.00000    0.00000   
             2017-10-04                250.07   22.0    1.00000    0.00000   
             2017-10-04                250.07   22.0    0.96987    0.01347   
             2017-10-06                250.07   24.0    1.00000    0.00000   
             2017-10-06                250.07   24.0    1.00000    0.00000   
             2017-10-06                250.07   24.0    0.95992    0.00421   
             2017-10-06                250.07   24.0    0.94834    0.00520   
             2017-10-06                250.07   24.0    1.00000    0.00000   
             2017-10-06                250.07   24.0    1.00000    0.00000   
             2017-10-06                250.07   24.0    1.00000    0.00000   
             2017-10-06                250.07   24.0    1.00000    0.00000   
             2017-10-06                250.07   24.0    0.94163    0.01132   
             2017-10-06                250.07   24.0    1.00000    0.00000   
             2017-10-06                250.07   24.0    0.00209    0.00108   
             2017-10-06                250.07   24.0    0.00255    0.00084   
             2017-10-06                250.07   24.0    0.00175    0.00070   
             2017-10-06                250.07   24.0    0.00217    0.00062   
             2017-10-06                250.07   24.0    0.00198    0.00052   
             2017-10-06                250.07   24.0    0.00148    0.00048   
             2017-10-11                250.07   29.0    1.00000    0.00000   
             2017-10-11                250.07   29.0    1.00000    0.00000   
             2017-10-11                250.07   29.0    1.00000    0.00000   
             2017-10-11                250.07   29.0    1.00000    0.00000   
             2017-10-11                250.07   29